# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 953.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 1.6 

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [4]:
# Data loading functions. Uncomment the one you want to use
#from worcgist.load_data import load_data
#from worclipo.load_data import load_data
from worcliver.load_data import load_data
#from ecg.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



The number of samples: 187
The number of columns: 494


In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold

# Pre processing
#
# Splitting

df_data = pd.DataFrame(data)

X = df_data.iloc[:, 1:]  # Feature set
y = df_data.iloc[:, :1]  # Classification Benign/Malignant

# Stratified K-Fold Split
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=40)

# Splitting data in Test and Train set
for train_index, test_index in skf.split(X, y):
    X_pre_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_pre_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Splitting data in Train and validation set
for train_index, test_index in skf.split(X_pre_train, y_pre_train):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

print(y_train)
print(f'The number of samples: {len(y_train)}')
print(f'test{y_test}')
print(f'The number of samples: {len(y_test)}')
print(f'val{y_val}')
print(f'The number of samples: {len(y_val)}')


                 label
ID                    
Liver-001_0     benign
Liver-004_0  malignant
Liver-005_0  malignant
Liver-006_0     benign
Liver-008_0  malignant
...                ...
Liver-143_0     benign
Liver-144_0  malignant
Liver-145_0     benign
Liver-146_0     benign
Liver-149_0  malignant

[120 rows x 1 columns]
The number of samples: 120
test                 label
ID                    
Liver-002_0     benign
Liver-003_0     benign
Liver-013_0     benign
Liver-024_0  malignant
Liver-034_0  malignant
Liver-038_0     benign
Liver-040_0     benign
Liver-045_0     benign
Liver-053_0     benign
Liver-061_0  malignant
Liver-067_0     benign
Liver-072_0  malignant
Liver-073_0  malignant
Liver-081_0  malignant
Liver-084_0     benign
Liver-091_0  malignant
Liver-095_0  malignant
Liver-096_0     benign
Liver-100_0     benign
Liver-104_0  malignant
Liver-107_0  malignant
Liver-111_0     benign
Liver-112_0  malignant
Liver-127_0     benign
Liver-131_0     benign
Liver-132_0  malignant
Li